In [1]:
from datasets import load_dataset

In [2]:
# Load English training split
train_dataset = load_dataset("ju-resplande/askD", split="train_en")

print(train_dataset[0])

Generating train_pt split: 0 examples [00:00, ? examples/s]

Generating train_en split: 0 examples [00:00, ? examples/s]

Generating validation_pt split: 0 examples [00:00, ? examples/s]

Generating validation_en split: 0 examples [00:00, ? examples/s]

Generating test_pt split: 0 examples [00:00, ? examples/s]

Generating test_en split: 0 examples [00:00, ? examples/s]

Generating external_pt split: 0 examples [00:00, ? examples/s]

Generating external_en split: 0 examples [00:00, ? examples/s]

{'q_id': '3yzn3p', 'title': 'Chest discomfort question', 'selftext': 'On and off for the last year or so, I\'ve been having occasional discomfort in my chest, particularly on the left side. The first time it happened, I was understandably worried and went to the emergency room. After some X-rays and an hour on an EKG, they sent me home saying they couldn\'t find evidence of anything wrong. Since then, it\'s happened several more times, occasionally accompanied by a feeling of shortness of breath. Whenever it happens, I check my pulse and nothing seems wrong (occasionally the pulse line looks a bit odd on my phone, but that could easily be an issue caused by my hand moving or something) and I never actually have any trouble drawing in breath.  Over the last few days, this has been recurring periodically, along with an occasional feeling of tightness in my throat and one or two bouts of transient weakness.\n\nAs background, I had two spontaneous pneumothoraxes in my left lung a little ov

In [4]:
train_dataset.to_parquet("../data/raw/askdocs/askdocs_train_en.parquet")


Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

45505564

In [ ]:
import pandas as pd

# Load Parquet
df = pd.read_parquet("../data/raw/askdocs/askdocs_train_en.parquet")


In [6]:
df.head()

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls
0,3yzn3p,Chest discomfort question,"On and off for the last year or so, I've been ...",,AskDocs,"{'a_id': ['cyi6ywp'], 'text': ['This could be ...",[],[],[]
1,3z0cuf,Firecracker exploded close to ear. Ears still ...,So i was messing around with firecrackers and ...,,AskDocs,"{'a_id': ['cyi9mz0', 'cyi8vl0'], 'text': ['I a...",[],[],[]
2,3z0mlc,If a Schizophrenic patient is positive-symptom...,Dear /r/AskDocs: I don't know such medicine-re...,,AskDocs,"{'a_id': ['cyic4j5', 'cyij9yl'], 'text': ['I d...",[],[],[]
3,3z0o23,Why all antipsychotics are more or less sedati...,Dear /r/AskDocs: I don't know such medicine-re...,,AskDocs,"{'a_id': ['cyihyla'], 'text': ['This is believ...",[],[],[http://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...
4,3z25e8,"I have a very embarrassing problem, if anyone ...","Hey, I'm a 19 year old male and I've had this ...",,AskDocs,"{'a_id': ['cyit9ci'], 'text': ['Pin worms? Goo...",[],[],[]


In [ ]:
import pandas as pd

In [28]:
result = pd.read_csv("../results/exp4_askdocs_gemini.csv", keep_default_na=False)

In [29]:
result.head()

,id,model,prediction,meta
0,9loutr,gemini,"Discovering a hard, disc-like area in your bre...","{""note"": ""real-world eval""}"
1,4fkkjd,gemini,"Yes, you should see a physician as soon as pos...","{""note"": ""real-world eval""}"
2,7m55bg,gemini,The oxygen gas itself does not expire or go ba...,"{""note"": ""real-world eval""}"
3,671fhy,gemini,I am incredibly sorry to hear about your broth...,"{""note"": ""real-world eval""}"
4,96n2tr,gemini,Based on the typical characteristics used in s...,"{""note"": ""real-world eval""}"


In [30]:
print(result["prediction"][0])

Discovering a hard, disc-like area in your breast can be caused by a number of conditions, and it is essential to have it evaluated by a healthcare professional. Benign (non-cancerous) possibilities include fibroadenomas, which are solid, rubbery lumps, cysts, which are fluid-filled sacs, or dense areas of tissue related to fibrocystic changes. However, a new, hard, or persistent lump can also be a sign of breast cancer. Because it is impossible to determine the cause without a physical examination and likely imaging tests like a mammogram or ultrasound, you should schedule an appointment with your doctor or a gynecologist as soon as possible. Prompt medical evaluation is the only way to get an accurate diagnosis, relieve your anxiety, and ensure you receive any necessary treatment.
